# Foundations of Data Science - Project 4 Brain Tumor Analysis

## Importing

### Importing libraries

In [17]:
import matplotlib as mpl
import pandas as pd
import seaborn as sns
import scipy as scs
import scipy.stats as sts

import zipfile
from zipfile import ZipFile

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import numpy as np 
from numpy import sqrt

import sklearn
from sklearn.metrics import roc_curve, confusion_matrix, auc, mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE, SelectKBest
from sklearn.datasets import load_iris
from sklearn import svm


In [5]:
#Check Version:
print("Python version:", "3.10.10")
print("Matplotlib version:", mpl.__version__)
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)
print("Seaborn version:", sns.__version__)
print("Scipy version:", scs.__version__)
print("Sklearn version:", sklearn.__version__)



Python version: 3.10.10
Matplotlib version: 3.7.1
Pandas version: 1.5.3
NumPy version: 1.23.5
Seaborn version: 0.12.2
Scipy version: 1.10.0
Sklearn version: 1.2.2


### Importing cvs files

In [18]:
clin_df_initial = pd.read_csv("../data/clinFeatures_UPENN.csv",index_col=False)

zf = zipfile.ZipFile("../data/radFeatures_UPENN.csv.zip") 
rad_df_initial = pd.read_csv(zf.open('radFeatures_UPENN.csv'), index_col=False)


clin_df = clin_df_initial
rad_df = rad_df_initial

## Data overview

In [19]:
print(clin_df.shape)
print(rad_df.shape)

(611, 10)
(611, 4753)


In [ ]:
print(clin_df.head(5))
print('----------------------------------')
print(rad_df.head(5))

### Overview of clinical data set

In [ ]:
#shape of data frame and data types
print(clin_df.info()) #patient numbers=611

print((clin_df.isna().sum(axis=1) == 0).sum()) #no patient with all attributes ->PsP_TP_score all (611) NaN 
print(clin_df['PsP_TP_score'].value_counts())

In [ ]:
print(clin_df.dtypes)

In [ ]:
# How many missing data is there in each row (axis = 1), default would be axis = 0
missing_values_1= clin_df.isna().sum(axis = 1) 
print(missing_values_1)

# How many missing data is there in each column (axis = 0)
missing_values_1= clin_df.isna().sum(axis = 0) 
print(missing_values_1)


In [ ]:
#brief summary of the extremes/means/medians & distribution
print(clin_df.describe())
print('MGMT:\n', clin_df['MGMT'].value_counts())
print('IDH1:\n', clin_df['IDH1'].value_counts())
print('KPS:\n', clin_df['KPS'].value_counts())
print('Time_since_baseline_preop:\n', clin_df['Time_since_baseline_preop'].value_counts())
print('Survival_from_surgery_days:\n', clin_df['Survival_from_surgery_days'].value_counts())

#Duplicate rows
print('Amount of duplicates:', clin_df.duplicated().sum())

In [ ]:
print('GTR_over90percent:\n',clin_df['GTR_over90percent'].value_counts()) #first considered as potential outcome
df_GTR = pd.DataFrame(clin_df['GTR_over90percent'])

### Overview of MRI data set 

In [ ]:
#Description of MRI data (=rad_df)
print(rad_df.shape)
print(rad_df.head(5))
print(rad_df.info())
print(rad_df.describe())

In [ ]:
#Check the data types
print(rad_df.dtypes)

In [ ]:
#Missing values for each row and column
missing_values_2= rad_df.isna().sum(axis = 1)
print(missing_values_2)

missing_values_2= rad_df.isna().sum(axis = 0)
print(missing_values_2)


In [ ]:
print(rad_df[rad_df.isna().any(axis=1)])

## Preprocessing 

In [ ]:
#Remove TP score column (because it's empty)
clin_df=clin_df.drop(columns=['PsP_TP_score'], axis=1) 
#check if the column has been removed correctly
print(clin_df.columns)

In [ ]:
#clean data
clin_df=clin_df[clin_df.Survival_from_surgery_days != 'Not Available']
print('Survival_from_surgery_days:\n', clin_df['Survival_from_surgery_days'].value_counts())
print(clin_df.shape)

In [ ]:
#adapt rad_df to processed clin data index
rad_df=rad_df.loc[clin_df.index]
print(rad_df.shape)

In [ ]:
#GTR numeric transformation (0=no, 1=yes, 2=Not available)
clin_df['GTR_over90percent']=pd.Categorical(clin_df['GTR_over90percent'], ordered=False)
print(clin_df.dtypes)
clin_df['GTR_over90percent'] = pd.factorize(clin_df['GTR_over90percent'])[0]
#print(clin_df)
print(clin_df['GTR_over90percent'].value_counts())

In [ ]:
#restrict analysis to complete cases
#print(rad_df.shape)
rad_df = rad_df[rad_df.isna().sum(axis=1)==0]
#print('missing values left?', rad_df[rad_df.isna().any(axis=1)])
#print(rad_df.shape) ##reduced from 452 to 312


In [ ]:
#saving the dataset after preprocessing for the visualisation
# Create a new dataset clin_df_vis with columns from clin_df
clin_df_vis = clin_df.assign(Gender=clin_df['Gender'], Age_at_scan_years=clin_df['Age_at_scan_years'])

# Print the resulting dataset
print(clin_df_vis)

In [ ]:
#Gender numeric transformation (M = 0 and F = 1)
clin_df['Gender'] = clin_df['Gender'].map({'F': 0, 'M': 1})

### Merging the two data sets 

In [ ]:
merged_df = pd.merge(clin_df, rad_df, on='SubjectID')
#print(merged_df.columns)
#print(merged_df.shape)
merged_df=merged_df.drop(columns=['SubjectID','Time_since_baseline_preop','KPS','Time_since_baseline_preop', 'IDH1', 'MGMT'], axis=1)

## Splitting Data

### Splitting into training and test set

In [ ]:
# Construct features and labels
df_X = merged_df.drop("Survival_from_surgery_days", axis=1)
df_y = merged_df["Survival_from_surgery_days"]

print(df_X)
print(df_y)

In [ ]:
# Split the dataset into training and testing sets using the sklearn 'train_test_split' function into 80% for training and 20% for testing,
# set the random seed to 2023.

df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size = 0.2, random_state = 2023)


#Scale the data

sc = StandardScaler()

df_X_train = sc.fit_transform(df_X_train)
df_X_test = sc.transform(df_X_test)

#convert it back into a pandas dataframe: 

df_X_train = pd.DataFrame(df_X_train, columns=df_X.columns)
df_X_test = pd.DataFrame(df_X_test, columns=df_X.columns)

# Data visualization 

In [ ]:
#Visualization: 

#custom colorblind friendly colors dictionary 
colorblind_15 = {
    # Source: http://mkweb.bcgsc.ca/biovis2012/color-blindness-palette.png
    'green':      [36, 255, 36],   #24ff24
    'orange':     [219, 109, 0],   #db6d00
    'lightblue':  [182, 219, 255], #b6dbff
    'darkblue':   [0, 109, 219],   #006ddb
    'violet':     [182, 109, 255], #b66dff
    'darkgreen':  [0, 73, 73],     #004949
    'brown':      [146, 73, 0],    #924900
    'purple':     [73, 0, 146],    #490092
    'red':        [146, 0, 0],     #920000
    'lightpink':  [255, 182, 219], #ffb6db
    'lightgreen': [0, 146, 146],   #009292
    'pink':       [255, 109, 182], #ff6db6
    'blue':       [109, 182, 255], #6db6ff
    'yellow':     [255, 255, 109], #ffff6d
    'black':      [0, 0, 0]        #000000
}
colorblind_normalized = {} 

for color, RGB in colorblind_15.items(): 
    # Get each RGB values
    r,g,b = colorblind_15[color]
    # Store the normalized values
    colorblind_normalized[color] = [r/255, g/255, b/255]
    
# Create a custom color palette compatible with seaborn library 
colorblind_palette_seaborn = colorblind_normalized

#set the default colorblind palette
sns.set_palette(colorblind_palette_seaborn)


In [ ]:
#color palette for report: black, malibu blue and silver
color_palette_bbg=['#000000', '#6db6ff', '#a3a3a3']

In [ ]:
#duplicates in clinical data? 
dup = clin_df_vis.duplicated().any()
print(dup)


### Age Distribution split by Gender

In [ ]:
# age distributed by gender: 
#difference before and after: 
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
age_gender_before = sns.histplot(clin_df_initial, x ='Age_at_scan_years', binwidth = 4, hue ='Gender', binrange= (18, 90), ax = axes[0])
age_gender_after = sns.histplot(clin_df_vis, x ='Age_at_scan_years', binwidth = 4, hue ='Gender', binrange= (18, 90), ax = axes[1])
label_axes_before = age_gender_before.set(xlabel='Age [year]', ylabel='Number of Patiens', title= 'Age distribution by Gender before Data Preprocessing') 
label_axes_after = age_gender_after.set(xlabel='Age [year]', ylabel='Number of Patiens', title= 'Age distribution by Gender after Data Preprocessing') 
plt.ylim(0, 50)

plt.savefig('../output/age_distribution_gender.png')


In [ ]:
# age distributed by gender: 
#difference before and after: 
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
age_gender_before = sns.histplot(clin_df_initial, x ='Age_at_scan_years', palette=color_palette_bbg[:2], binwidth = 4, hue ='Gender', binrange= (18, 90), ax = axes[0])
age_gender_after = sns.histplot(clin_df_vis, x ='Age_at_scan_years', palette=color_palette_bbg[:2],binwidth = 4, hue ='Gender', binrange= (18, 90), ax = axes[1])
label_axes_before = age_gender_before.set(xlabel='Age [year]', ylabel='Number of Patiens', title= 'Age distribution by Gender before Data Preprocessing') 
label_axes_after = age_gender_after.set(xlabel='Age [year]', ylabel='Number of Patiens', title= 'Age distribution by Gender after Data Preprocessing') 
plt.ylim(0, 50)

plt.savefig('../output/age_distribution_gender_blackblue.png')


### Survival days based on Age and Gender

In [ ]:
#Box plots for survival days (x-axis) and age (y-axis) and gender (hue)
a = np.arange(0, 2208)
#print(a)
bins = 10
_, b = np.histogram(a, bins)
#print(b)
c = np.array(merged_df['Survival_from_surgery_days']).astype(int)
_, d = np.histogram(c, bins)
indices = np.digitize(c, d)
#print(indices)
#print(b)


plt.figure(figsize=(12, 8))
fig_surv_age = sns.boxplot(x = indices, y = merged_df['Age_at_scan_years'], hue=merged_df['Gender'], width=0.4)
#fig_label =fig_surv_age.set(xlabel='Survival Rate [days]', ylabel='Age [year]', title= 'Age Range split by Gender and Outcome')
plt.xlabel('Survival Rate [days]', fontsize=14)
plt.ylabel('Age [year]', fontsize=14)
plt.title('Age Range split by Gender and Survival days', fontsize=17)
custom_labels = ['0-220', '221-441', '442-662', '663-882', '883-1103', '1104-1324', '1325-1544', '1545-1765', '1766-1986', '1987-2207']
fig_surv_age.set_xticklabels(custom_labels)


#Modify legend: 
unique_values = clin_df['Gender'].unique()
#Create custom legend handles and labels
custom_handles = [plt.Rectangle((0, 0), 1, 1, fc=color)  for color in sns.color_palette(n_colors=len(unique_values))]

custom_labels = ['F', 'M']
fig_surv_age.legend(custom_handles, custom_labels, title='Gender')

#Aesthetics: Black frame around the colours of the legend
legend = plt.legend(handles=custom_handles, labels=custom_labels, title='Gender')
for handle in legend.legendHandles:
    handle.set_linewidth(1)
    handle.set_edgecolor('black')



plt.savefig('../output/boxplot_survival_age_gender')

In [ ]:
#Box plots for survival days (x-axis) and age (y-axis) and gender (hue)
a = np.arange(0, 2208)
#print(a)
bins = 10
_, b = np.histogram(a, bins)
#print(b)
c = np.array(merged_df['Survival_from_surgery_days']).astype(int)
_, d = np.histogram(c, bins)
indices = np.digitize(c, d)
#print(indices)
#print(b)


plt.figure(figsize=(12, 8))
fig_surv_age = sns.boxplot(x = indices, y = merged_df['Age_at_scan_years'], hue=merged_df['Gender'], palette=color_palette_bbg[:2], width=0.4)
#fig_label =fig_surv_age.set(xlabel='Survival Rate [days]', ylabel='Age [year]', title= 'Age Range split by Gender and Outcome')
plt.xlabel('Survival Rate [days]', fontsize=14)
plt.ylabel('Age [year]', fontsize=14)
plt.title('Age Range split by Gender and Survival days', fontsize=17)
custom_labels = ['0-220', '221-441', '442-662', '663-882', '883-1103', '1104-1324', '1325-1544', '1545-1765', '1766-1986', '1987-2207']
fig_surv_age.set_xticklabels(custom_labels)


#Modify legend: 
unique_values = clin_df['Gender'].unique()
#Create custom legend handles and labels
custom_handles = [plt.Rectangle((0, 0), 1, 1, fc=color)  for color in  sns.color_palette(color_palette_bbg[:2])]
custom_labels = ['F', 'M']
fig_surv_age.legend(custom_handles, custom_labels, title='Gender')

#Aesthetics: Black frame around the colours of the legend
legend = plt.legend(handles=custom_handles, labels=custom_labels, title='Gender')
for handle in legend.legendHandles:
    handle.set_linewidth(1)
    handle.set_edgecolor('black')



plt.savefig('../output/boxplot_survival_age_gender_blackblue')


# Correlations between Variables  

## Test for normality and correlations

In [ ]:
correlation_matrix = df_X_train.corr(method ='spearman')
#print(correlation_matrix.shape)
#print(correlation_matrix)

In [ ]:
# coefficient matrix upper triangle
#indices = np.triu_indices(correlation_matrix.shape[0], k=1)
#array_correlation_matrix = correlation_matrix[indices]

#print(array_correlation_matrix)

# Exclude duplicate and diagonal values
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k = 1)
#mask = np.triu_indices_from(correlation_matrix, k = 1)

correlation_matrix_clean = correlation_matrix.where(mask)
print()

# Flatten the correlation matrix (excluding NaN values)
correlation_values = correlation_matrix_clean.stack().values

# Plot the distribution of correlation coefficients
sns.histplot(correlation_values, kde=True, color = color_palette_bbg[2])
plt.xlabel('Correlation Coefficient')
plt.ylabel('Frequency')
plt.title('Distribution of Correlation Coefficients')

plt.savefig('../output/distribution_of_correlation_coefficients.png') 

plt.show()


In [ ]:
print(correlation_matrix_clean)
print('******************************')
print(correlation_values)
print('******************************')
print(correlation_values.shape)

In [ ]:
data = df_X_train
print(data.shape)

In [ ]:
print(correlation_matrix.shape)

In [ ]:
#dropping one of the features with a correlation higher than +/-0.7 (based on literature)
correlation_threshold = 0.7
dropped_features = set()


for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if np.sqrt((correlation_matrix.iloc[i, j])**2) >= (correlation_threshold):
            colname_i = correlation_matrix.columns[i]
            colname_j = correlation_matrix.columns[j]
            if colname_i not in dropped_features:
                dropped_features.add(colname_j)
                

# Drop the correlated features from the dataset
data.drop(dropped_features, axis = 1, inplace=True)

# Print the updated dataset
print(data.shape)



In [ ]:
df_X_test.drop(dropped_features, axis = 1, inplace=True)

## Feature selection

### Univariate Feature selection (Recursive Feature Eliminiation)

In [ ]:
#univariate feature selection -> RFE: 
data = pd.DataFrame(data)
df_y_train = pd.Series(df_y_train) 

estimator = LinearRegression()
rfe = RFE(estimator=estimator, n_features_to_select=236)
rfe.fit(data, df_y_train)
selected_features = rfe.support_

### Comparison: Mutual Info Regression

In [ ]:
# Models performed better with RFE -> used mututal_info_regression for the comparison

"""
mutual_info = mutual_info_regression(data, df_y_train)

#Representing in list form
mutual_info = pd.Series(mutual_info)
mutual_info.index = data.columns
#print(mutual_info.sort_values(ascending=False))

#plotting
mutual_info.sort_values(ascending=False).plot.bar(figsize=(20, 8))
mutual_info_not=(mutual_info.values<0.001).sum()

#print(mutual_info_not)
k=741-mutual_info_not
print(k)

#Selecting best k features

from sklearn.feature_selection import SelectKBest

#selecting the top important features
sel_best_cols = SelectKBest(mutual_info_regression, k=k)
sel_best_cols.fit(data, df_y_train)
selected_features_columns = data.columns[sel_best_cols.get_support()]
"""
#k=236 -> used also 236 features for the RFE

In [ ]:
#print(selected_features)

In [ ]:
selected_features_columns = data.columns[selected_features]

In [ ]:
print(selected_features_columns)
print(selected_features_columns.shape)

In [ ]:
data_final_features = data[selected_features_columns]

In [ ]:
print(data_final_features)

## Adjusting the test data: need the same features

In [ ]:
df_X_test_final_features = df_X_test[selected_features_columns]


In [ ]:
print(df_X_test_final_features)

In [ ]:
print(data.shape)
print(df_y_train.shape)

## Machine Learning Models with cross validation

### Evaluation metrics

In [ ]:
    
def get_scores(model, data_final_features, df_y_train, df_X_test, df_y_test):
    y_pred_test = model.predict(df_X_test)
    y_pred_train = model.predict(data_final_features)
    
    # evaluation
    #r2_test = r2_score(y_test, y_pred_test)
    #rmse_test = mean_squared_error(y_test, y_pred_test, squared=True)
    r2_test = r2_score(df_y_test, y_pred_test)
    rmse_test = mean_squared_error(df_y_test, y_pred_test, squared=True)

    #r2_train = r2_score(y_train, y_pred_train)
    #rmse_train = mean_squared_error(y_train, y_pred_train, squared=True)
    r2_train = r2_score(df_y_train, y_pred_train)
    rmse_train = mean_squared_error(df_y_train, y_pred_train, squared=True)


    print('Training set score: R2 score: {:.3f}, RMSE: {:.3f}'.format(r2_train, rmse_train))
    print('Test set score: R2 score: {:.3f}, RMSE: {:.3f}'.format(r2_test, rmse_test))


############################################################################################################

def evaluation_metrics(clf, y, X):
    # Get the predicted labels
    y_test_pred = clf.predict(X)

    # Calculate the evaluation metrics for regression
    r2 = r2_score(y, y_test_pred)
    mse = mean_squared_error(y, y_test_pred)
    rmse = sqrt(mse) 
    mae = mean_absolute_error(y, y_test_pred)
    
    return [r2, rmse, mae]

### Crossvalidation

In [ ]:
# Perform a 3-fold cross-validation

# Set the random seed
random_seed = 42
np.random.seed(random_seed)

# Initialize the cross-validator
n_splits = 3
kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_seed)

# Prepare the performance overview DataFrame
df_performance = pd.DataFrame(columns=['fold', 'clf', 'r2', 'rmse', 'mae'])

# Use this counter to save your performance metrics for each cross-validation fold
fold_number = 0
for train_index, test_index in kf.split(data_final_features):
    
    x_train_cv = data_final_features.iloc[train_index] 
    x_val_cv = data_final_features.iloc[test_index]
    y_train_cv = df_y_train.iloc[train_index]
    y_val_cv = df_y_train.iloc[test_index]
    
    # Linear regression
    clf = LinearRegression()
    clf.fit(x_train_cv, y_train_cv)
    eval_metrics_LR = evaluation_metrics(clf, y_val_cv, x_val_cv)
    df_performance.loc[len(df_performance)-1, :] = [fold_number, 'LinearRegression'] + eval_metrics_LR

    # Lasso regression
    clf_lasso = Lasso(random_state=random_seed)
    clf_lasso.fit(x_train_cv, y_train_cv)
    eval_metrics_lasso = evaluation_metrics(clf_lasso, y_val_cv, x_val_cv)
    df_performance.loc[len(df_performance)-1, :] = [fold_number, 'LassoRegression'] + eval_metrics_lasso

    # Ridge regression
    clf_ridge = Ridge(random_state=random_seed)
    clf_ridge.fit(x_train_cv, y_train_cv)
    eval_metrics_ridge = evaluation_metrics(clf_ridge, y_val_cv, x_val_cv)
    df_performance.loc[len(df_performance)-1, :] = [fold_number, 'RidgeRegression'] + eval_metrics_ridge

    # Random forest
    clf_rf = RandomForestRegressor(random_state=random_seed)
    clf_rf.fit(x_train_cv, y_train_cv)
    eval_metrics_rf = evaluation_metrics(clf_rf, y_val_cv, x_val_cv)
    df_performance.loc[len(df_performance)-1, :] = [fold_number, 'RandomForest'] + eval_metrics_rf
    
    #SVM
    clf_svm = svm.SVR(kernel='rbf')
    clf_svm.fit(x_train_cv, y_train_cv)
    eval_metrics_svm = evaluation_metrics(clf_svm, y_val_cv, x_val_cv)
    df_performance.loc[len(df_performance)-1, :] = [fold_number, 'SupportVectorMachine'] + eval_metrics_svm    

    # Increase counter for folds
    fold_number += 1

# Print the performance overview DataFrame
print(df_performance)


In [ ]:
df_performance_mean = df_performance.groupby('clf').mean()
print(df_performance_mean)

## Final Feature selection

In [ ]:
coefficients = clf_ridge.coef_
abs_coefficients = abs(coefficients)


# Compute the normalized feature importance
feature_importance = abs_coefficients / abs_coefficients.sum()

# Create a dictionary to store the feature importance
feature_importance_dict = dict(zip(data_final_features.columns, feature_importance))

# Sort the feature importance dictionary by values in descending order
sorted_feature_importance = dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True))



# Get all features
all_indices = abs_coefficients.argsort()[:][::-1]

all_features = [data_final_features.columns[i] for i in all_indices]


# Get the top 10 features based on the absolute coefficient values
top_10_indices = abs_coefficients.argsort()[-10:][::-1]

top_10_features = [data_final_features.columns[i] for i in top_10_indices]



In [ ]:
# Get all features
all_indices = abs_coefficients.argsort()[:][::-1]

all_features = [data_final_features.columns[i] for i in all_indices]



# All features plot
feature_importance_df_all = pd.DataFrame(columns=['Feature', 'Importance'])

# Iterate over the top_10_features list
for feature in all_features:
    importance_all = sorted_feature_importance.get(feature)
    feature_importance_df_all = feature_importance_df_all.append({'Feature': feature, 'Importance': importance_all}, ignore_index=True)

# Print the feature importance DataFrame
print(feature_importance_df_all)


feature_importance_all_plot = sns.barplot(feature_importance_df_all, x = 'Feature', y = 'Importance', color = color_palette_bbg[2])
# feature_importance_all_plot.set_xticklabels(feature_importance_all_plot.get_xticklabels(), rotation=90)
plt.xticks([])
plt.savefig('../output/feature_importance_all')



In [ ]:
#Top 10 features plot

feature_importance_df_top_10 = pd.DataFrame(columns=['Feature', 'Importance'])

# Iterate over the top_10_features list
for feature in top_10_features:
    importance_top_10 = sorted_feature_importance.get(feature)
    feature_importance_df_top_10 = feature_importance_df_top_10.append({'Feature': feature, 'Importance': importance_top_10}, ignore_index=True)

# Print the feature importance DataFrame
print(feature_importance_df_top_10)

feature_importance_top_10_plot = sns.barplot(feature_importance_df_top_10, x = 'Feature', y = 'Importance', color = color_palette_bbg[2])
feature_importance_top_10_plot.set_xticklabels(feature_importance_top_10_plot.get_xticklabels(), rotation=90)
plt.ylim(0.008, 0.014)

plt.xlabel('Feature Names', fontsize=12)
plt.ylabel('Absolute Importance of Top Ten Features', fontsize=12)
plt.title('Top Ten Features', fontsize=16)
plt.ylim(0.01, 0.0135)

plt.savefig('../output/top10_feature_importance')


### Top Ten Features Linear?

In [ ]:
top10_features_names = feature_importance_df_top_10['Feature']
#print(top10_features_names)
top10_features_list = top10_features_names.tolist()
top10_features_data = pd.merge(merged_df[top10_features_list], merged_df['Survival_from_surgery_days'], left_index=True, right_index=True)
top10_features_data['Survival_from_surgery_days'] = pd.to_numeric(top10_features_data['Survival_from_surgery_days'])
top10_features_data = top10_features_data.sort_values(by='Survival_from_surgery_days', ascending = True)
print(top10_features_data)

import matplotlib.ticker as mticker

fig, axes = plt.subplots(5, 2, figsize=(14, 14))
axes = axes.flatten()
for i, feature in enumerate(top10_features_list):
    ax = sns.scatterplot(data = top10_features_data, x = feature , y = 'Survival_from_surgery_days', ax = axes[i], color = color_palette_bbg[2])
    ax.set_xlabel(feature)
    ax.set_ylabel('Survival days')
    ax.yaxis.set_major_locator(mticker.MultipleLocator(200))

    
fig.suptitle('Linearity of the Top Ten Features', fontsize=14)
plt.tight_layout()

plt.savefig('../output/Linearity_TopTenFeatures.png')

plt.show()



## Evaluation of the final model (Ridge Regression)

In [ ]:
#compute model with our training set and predict it for the test set
clf_ridge.fit(data_final_features, df_y_train)
coefficients_RidgeR = clf_ridge.coef_
intercept_RidgeR = clf_ridge.intercept_
y_pred_test_RidgeR = clf_ridge.predict(df_X_test_final_features)

# Calculate the evaluation metrics for regression
r2_RidgeR = r2_score(df_y_test, y_pred_test_RidgeR)
mse_RidgeR = mean_squared_error(df_y_test, y_pred_test_RidgeR)
rmse_RidgeR = sqrt(mse_RidgeR)
mae_RidgeR = mean_absolute_error(df_y_test, y_pred_test_RidgeR)

df_performance_RidgeR = {'R2': r2_RidgeR, 'RMSE': rmse_RidgeR, 'MAE': mae_RidgeR}

print(df_performance_RidgeR)
